In [1]:
# Import the modules
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA

In [6]:
# Read the csv file into a pandas DataFrame
scaled_df = pd.read_csv(
    Path("../Data/scaled_df.csv")
)

# Review the DataFrame
scaled_df.head()

,LoanStatus,Term,ProsperRating (numeric),Average Credit Score,EmploymentStatus,IsBorrowerHomeowner,StatedMonthlyIncome,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,RevolvingCreditBalance,LoanOriginalAmount
0,-0.331641,-0.557261,1.152074,-0.411842,-0.328162,-1.058752,0.023500,-1.393753,-0.608327,-1.152967,0.144557,-0.444937,0.145693
1,-0.331641,-0.557261,1.152074,2.136159,-0.328162,0.944508,-0.370888,-1.321398,0.102440,-0.001558,0.158056,-0.526144,0.145693
2,-0.331641,-0.557261,-0.640879,-0.411842,-0.328162,0.944508,0.443168,0.167245,-0.608327,-0.721189,1.457209,-0.374610,0.940903
3,-0.331641,1.504434,0.554423,0.862158,-0.328162,0.944508,0.291481,-0.865828,-0.216890,-0.937078,0.270123,1.437989,0.940903
4,-0.331641,-0.557261,-1.238529,-0.411842,-0.328162,-1.058752,-0.466957,1.007371,0.710197,-0.361374,-0.906786,-0.386768,-0.967602


In [7]:
# Instantiate the PCA instance and declare the number of PCA variables
pca=PCA(n_components=2)

In [8]:
# Fit the PCA model on the transformed credit card DataFrame
customers_pca = pca.fit_transform(scaled_df)

# Review the first 5 rows of the array of list data
customers_pca[:5]

array([[ 1.10387539, -0.33762012],
       [ 1.90506764, -1.88229392],
       [ 0.84163516,  1.17573644],
       [ 2.29100065,  0.57309442],
       [-1.98928313,  0.65614495]])

In [9]:
# Calculate the PCA explained variance ratio
pca.explained_variance_ratio_

array([0.26295323, 0.12703173])

In [10]:
# Create the PCA DataFrame
customers_pca_df = pd.DataFrame(
    customers_pca,
    columns=["PCA1", "PCA2"]
)

# Review the PCA DataFrame
customers_pca_df.head()

,PCA1,PCA2
0,1.103875,-0.337620
1,1.905068,-1.882294
2,0.841635,1.175736
3,2.291001,0.573094
4,-1.989283,0.656145


In [11]:
# Create a a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

In [12]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the service_ratings DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(customers_pca_df)
    inertia.append(k_model.inertia_)

In [13]:
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}

# Create the DataFrame from the elbow data
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,430091.791831
1,2,234847.402385
2,3,166383.760684
3,4,125080.683280
4,5,107450.588099


In [14]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

In [15]:
# Define the model Kmeans model using the optimal value of k for the number of clusters.
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(customers_pca_df)

# Make predictions
k_3 = model.predict(customers_pca_df)

# Create a copy of the customers_pca_df DataFrame
customer_pca_predictions_df = customers_pca_df.copy()

# Add a class column with the labels
customer_pca_predictions_df["customer_segments"] = k_3

In [16]:
# Plot the clusters
customer_pca_predictions_df.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="customer_segments"
)

:NdOverlay   [customer_segments]
   :Scatter   [PCA1]   (PCA2)

In [19]:
# Define the model Kmeans model using k=3 clusters
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(scaled_df)

# Make predictions
k_3 = model.predict(scaled_df)

# Create a copy of the customers_transformed_df DataFrame
scaled_predictions_df = customers_transformed_df.copy()

# Add a class column with the labels
scaled_predictions_df["customer_segments"] = k_3

In [20]:
# Plot the clusters using the first two feature columns
customers_transformed_predictions_df.hvplot.scatter(
    x="feature_1",
    y="feature_2",
    by="customer_segments"
)

DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['feature_1', 'feature_2']

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html